In [17]:
import os
import json
from recordtype import recordtype
import numpy as np
import dadrah.util.data_processing as dapr

## params

In [10]:
Parameters = recordtype('Parameters','qr_run_n, sig_sample_id, sig_xsec, poly_order, kfold_n')
params = Parameters(qr_run_n = 31, sig_sample_id='GtoWW35naReco', sig_xsec=0, poly_order=5, kfold_n=5) 
quantiles = [0.3, 0.5, 0.7, 0.9]

## paths

In [11]:
envelope_dir = '/eos/user/k/kiwoznia/data/QR_results/analysis/vae_run_113/qr_run_'+str(params.qr_run_n)+'/sig_GtoWW35naReco/xsec_'+str(params.sig_xsec)+'/loss_rk5_05/envelope'
polys_json_path = os.path.join(envelope_dir, 'polynomials_allQ_allFolds_'+ params.sig_sample_id + '_xsec_' + str(params.sig_xsec) + '.json')

## read cuts

In [18]:
envelope_per_fold = {}
for k in range(params.kfold_n+1):
    envelope_json_path = os.path.join(envelope_dir, 'cut_stats_allQ_fold'+str(k+1)+'_'+ params.sig_sample_id + '_xsec_' + str(params.sig_xsec) + '.json')
    ff = open(envelope_json_path)
    envelope_per_fold['fold_{}'.format(k+1)] = json.load(ff)

In [19]:
envelope_per_fold

{'fold_1': {'q30': [[1227.5,
    1.6780740320682526,
    0.00015267097399362296,
    1.6778203248977661,
    1.6781971454620361],
   [1287.5,
    1.7057816684246063,
    0.00015420470099296256,
    1.7056163549423218,
    1.7059500217437744],
   [1353.5,
    1.7364886105060577,
    0.0003911997406760705,
    1.7358297109603882,
    1.736797571182251],
   [1422.0,
    1.768930196762085,
    0.0003906110892127425,
    1.7685075998306274,
    1.7694745063781738],
   [1493.0,
    1.802453190088272,
    0.0005055606264713822,
    1.8018585443496704,
    1.8031853437423706],
   [1566.5,
    1.8381820619106293,
    0.00048120009457189886,
    1.8374121189117432,
    1.8386698961257935],
   [1642.5,
    1.8758427798748016,
    5.4176286000878055e-05,
    1.87575364112854,
    1.8758916854858398],
   [1721.0,
    1.916513442993164,
    0.0002442123699522425,
    1.9161221981048584,
    1.9167379140853882],
   [1802.5,
    1.9601280093193054,
    0.0005448786467886384,
    1.9593641757965088,
  

## read polynomials

In [12]:
print('reading polynomials from ' + polys_json_path)
polynomials_folds = dapr.read_polynomials_from_json(polys_json_path, quantiles, params.kfold_n)

reading polynomials from /eos/user/k/kiwoznia/data/QR_results/analysis/vae_run_113/qr_run_31/sig_GtoWW35naReco/xsec_0/loss_rk5_05/envelope/polynomials_allQ_allFolds_GtoWW35naReco_xsec_0.json


In [13]:
polynomials_folds

{'fold_1': {0.3: poly1d([ 1.57289285e-19, -1.10312403e-14,  1.24951203e-10, -3.40444832e-07,
          8.05580958e-04,  9.95896469e-01]),
  0.5: poly1d([-1.92347335e-19, -6.07886919e-15,  9.91775910e-11, -2.31456067e-07,
          6.27690643e-04,  1.14711002e+00]),
  0.7: poly1d([-4.82054487e-19, -3.27786814e-15,  8.39862808e-11, -1.38987933e-07,
          4.70606210e-04,  1.28102294e+00]),
  0.9: poly1d([-2.74872298e-18,  2.99212609e-14, -9.48545006e-11,  3.82399092e-07,
         -1.81153638e-04,  1.64305884e+00])},
 'fold_2': {0.3: poly1d([ 9.37525299e-20, -1.02749124e-14,  1.22036668e-10, -3.36108462e-07,
          8.03973636e-04,  9.95083800e-01]),
  0.5: poly1d([-1.94037693e-19, -6.74837858e-15,  1.05434938e-10, -2.52221303e-07,
          6.56752875e-04,  1.13262209e+00]),
  0.7: poly1d([-1.01816773e-18,  4.46820251e-15,  4.23486972e-11, -3.51162441e-08,
          3.50179006e-04,  1.33323454e+00]),
  0.9: poly1d([-2.44241674e-18,  2.37631349e-14, -5.45149393e-11,  2.68163319e-07,


## check fitting

In [20]:
# check for each fold
for k in range(params.kfold_n+1):
    poly = polynomials_folds['fold_{}'.format(k+1)]
    envelope = envelope_per_fold['fold_{}'.format(k+1)]
    for q in quantiles:
        poly_q = poly[q]
        envelope_q = np.asarray(envelope[q])

KeyError: 0.3